In [1]:
import pandas as pd
%matplotlib inline
import matplotlib
matplotlib.style.use('ggplot')

As you may have seen, I'm back through 2008 on the Big 3 (Governor's Ball has only been multiple days for the last 4 years, so I stopped after that). I'd like to do a graph over time of the % of unique bands for each festival. I don't know if there's a quick code way to do that, but I can also just count off the spreadsheet. I'm going to go back to 2005, when Lolla started as a destination fest.

It might also be good to re-do the 2016 graph as "% unique bands" instead of "% appearing at another fest". Just seems more intuitive to flip it.

I'd also like to see how that "% unique bands" graph changes if we cut it off at top 25, top 50, top 75, top 100. 

Are these doable? I'll use those results for the pitch.


In [13]:
df2016 = pd.read_excel('/Users/hunterowens/Downloads/Festivals.xlsx')

In [14]:
# Data clean and setup
df2016.drop([0,1], inplace=True)
df2016.drop(['AVG RANK','# OF FESTS', 'PWR RANKING', 'Unnamed: 4'], axis=1)
df2016.set_index('ARTIST', inplace=True)

In [15]:
df2016

,AVG RANK,# OF FESTS,PWR RANKING,Unnamed: 4,COACHELLA,GOVBALL,BONNAROO,JAZZFEST,SUNFEST,SHAKYKNEES,...,WAYHOME,LOLLA,OSHEAGA,OUTSIDELDS,FYF,Unnamed: 24,PITCHFORK,BUMBERSHT,ACL,EAUXCLAIRE
ARTIST,,,,,,,,,,,,,,,,,,,,,
1200,37.000000,1,37.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2 Many DJs,14.000000,1,14.000000,NaN,14,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3LAU,24.000000,1,24.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
A Tribe Called Red,29.000000,1,29.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
A$AP Ferg,54.000000,1,54.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
A$AP Rocky,6.250000,4,1.562500,NaN,5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
Aaron Neville,33.000000,1,33.000000,NaN,NaN,NaN,NaN,33,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
AC Slater,80.000000,2,40.000000,NaN,50,NaN,NaN,NaN,NaN,NaN,...,NaN,110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
Action Bronson,22.000000,1,22.000000,NaN,NaN,22,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
